In [1]:
import pandas as pd
import geopandas as gpd

import plotly.express as px
import pydeck as pdk

In [2]:
corresps = {}
df = pd.read_csv(r'../data/corresps/corresp-nuts.csv')
corresps['nuts'] = df.sort_values(['year', 'old']).groupby('old')['new'].last().to_dict()

In [3]:
epsgs = {'world': 4326, 'proj': 32633}

In [4]:
main_greek_ports = ['GRGPA', 'GRPIR', 'GRIGO', 'GRSKG', 'GRLVR']

In [5]:
nuts = gpd.read_file(r'../data/boundaries/NUTS_RG_03M_2021_4326.json', ).set_index('id')
#nuts = nuts[nuts.LEVL_CODE==3]
nuts = nuts.set_crs(epsg=4326).to_crs(epsg=epsgs['proj'])

#buffer to ensure inclusion of ports
nuts.geometry = nuts.buffer(200)

nuts_centroids = nuts.copy()
nuts_centroids.geometry = nuts_centroids.geometry.centroid
nuts.head(2)

,NUTS_ID,LEVL_CODE,CNTR_CODE,NAME_LATN,NUTS_NAME,MOUNT_TYPE,URBN_TYPE,COAST_TYPE,FID,geometry
id,,,,,,,,,,
BG,BG,0,BG,Bulgaria,България,0.0,NaN,0,BG,"POLYGON ((1143353.225 4881912.962, 1147981.182..."
CH,CH,0,CH,Schweiz/Suisse/Svizzera,Schweiz/Suisse/Svizzera,0.0,NaN,0,CH,"POLYGON ((21924.555 5314166.492, 21943.141 531..."


In [6]:
ports = gpd.read_file(r'../data/PORT_2013_SH/Data/PORT_PT_2013.shp')
ports = ports.to_crs(epsg=epsgs['proj'])

ports = ports.rename(columns={'PORT_ID': 'id'}).set_index('id')

# Some ports are not placed within the respective nuts
# ports.geometry = ports.buffer(200)

ports = (ports.sjoin(nuts.reset_index()[['id', 'geometry']], predicate='within')
              .rename(columns={'id': 'nuts'}).drop(columns=['index_right']))

ports['country'] = ports['nuts'].str[0:2]

cols = ['nuts', 'country', 'geometry']
ports = ports[cols]

# Keep only the main greek ports
g = ports.loc[main_greek_ports]
mask = ports['country']=='EL'
ports = pd.concat([ports.drop(ports[mask].index), g])
ports.head(2)

,nuts,country,geometry
id,,,
FRSXB,FR,FR,POINT (-34261.191 5405630.512)
FRYNE,FR,FR,POINT (-242083.058 4812559.159)


In [7]:
# Identify the catchment area
dfg = ports.copy()
dfg.geometry = dfg.buffer(150_000)
catcharea = gpd.sjoin(dfg, nuts_centroids, predicate='contains').groupby(level='id')['FID'].agg(tuple)
catcharea.loc[main_greek_ports]

id
GRGPA    (EL65, EL65, EL65, EL65, EL651, EL651, EL651, ...
GRPIR    (EL65, EL65, EL65, EL65, EL651, EL651, EL651, ...
GRIGO    (EL622, EL622, EL622, EL622, AL035, AL035, AL0...
GRSKG    (EL515, EL515, EL515, EL515, EL61, EL61, EL61,...
GRLVR    (EL652, EL652, EL652, EL652, EL306, EL306, EL3...
Name: FID, dtype: object

In [8]:
tmp = pd.read_excel(r'../data/data.xlsx', sheet_name=list(range(7)))
lookups = {}
for m in tmp:
    df = tmp[m]
    df = df.set_index(df.columns[0]).squeeze()
    lookups[tmp[m].columns[0]] = df.to_dict()

In [9]:
try:
    data = pd.read_pickle(r'../data/data.pkl')
except:
    data = pd.read_excel(r'../data/data.xlsx', sheet_name='DB')
    #data = data.replace(namemaps)
    data.to_pickle(r'../data/data.pkl')

In [10]:
df = data.copy()
df["combined"] = ~df["origin_port"].isna()        

df['origin_nuts'] = df['origin_nuts'].replace(lookups['nuts'])
df['destination_nuts'] = df['destination_nuts'].replace(lookups['nuts'])

# df = df.replace("^GR", "EL", regex=True)
df["origin_country"] = df["origin_nuts"].str[0:2]
df["destination_country"] = df["destination_nuts"].str[0:2]

nuts_to_ports = ports.reset_index().set_index("nuts")["id"].to_dict()
df["origin_port"] = df["origin_port"].replace(lookups['nuts']).replace(nuts_to_ports)
df["destination_port"] = df["destination_port"].replace(lookups['nuts']).replace(nuts_to_ports)

for col in ['carriage_type', 'package_type', 'cargo_type', 'cargo_group_type', 'vehicle_type']:
    df[col] = df[col].replace(lookups[col])
    df[col] = df[col].astype('category')

df.to_pickle('../data/data-clean.pkl')

In [ ]:
combined_trans = df[df['combined']]
combined_trans.head(2)

In [ ]:
dff = combined_trans.copy()
dff = dff[dff.origin_country=='EL']
dff = dff[dff.destination_country!='EL']
#dff['origin_nuts'] = dff['origin_nuts'].replace(lookups['Nuts-name'])
#dff['destination_nuts'] = dff['destination_nuts'].replace(lookups['Nuts-name'])
#dff = dff.groupby(['origin_port', 'origin_nuts']).size().unstack()
#dff.loc['GRGPA'].dropna()

In [ ]:
dff.origin_nuts

In [ ]:
nuts.loc['EL3']

In [ ]:
lvl = 1

dff = combined_trans.copy()
dff = dff[dff.origin_country=='EL']
dff = dff[dff.destination_country!='EL']
dff = dff[dff.origin_port=='GRGPA']

dff.origin_nuts = dff.origin_nuts.str[:-3+lvl]
dff.origin_nuts

In [ ]:
dff = combined_trans.copy()
dff = dff[dff.origin_country=='EL']
dff = dff[dff.destination_country!='EL']
dff = dff[dff.origin_port=='GRGPA']

dff.origin_nuts = dff.origin_nuts.str[:4] 
dff = dff.groupby(['destination_nuts'])['loaded_weight_kg'].size()

n = nuts.copy()
dfp = n.join(dff, how='right')
dfp = dfp.to_crs(epsg=epsgs['world'])

bbox = dfp.total_bounds
center = {'lat': (bbox[1]+bbox[3])/2, 'lon': (bbox[0]+bbox[2])/2}

dfp.ori
px.choropleth_mapbox(dfp, geojson=dfp.geometry, locations=dfp.index,
                           color='loaded_weight_kg',
                           center=center,
                           color_continuous_scale=px.colors.sequential.Reds[1:],
                           mapbox_style='carto-positron')

In [ ]:
dfp.groupby('CNTR_CODE')['loaded_weight_kg'].sum()

In [ ]:
dff = df[df.destination_country!='EL']
dff

In [ ]:
df = combined_trans.copy()
df

In [ ]:
port = 'GRGPA'
catchareaf = catcharea.loc[port:port]
catchareaf

In [ ]:
# Filter the combined transports based on the 75/130/EEC directive 
mask = df[['origin_port', 'origin_nuts']].apply(lambda x: x['origin_nuts'] in catchareaf.get(x['origin_port'], []), axis=1)
dff = df[mask]
dff

In [ ]:
vols = dff.groupby(['origin_nuts', 'destination_nuts'])['loaded_weight_kg'].sum()
origs = vols.reset_index().set_index('origin_nuts').join(nuts_centroids['geometry']).rename(columns={'geometry': 'orig_geom'})
origs.index.name = 'origin_nuts'
#dests = vols.reset_index().set_index('destination_nuts').join(nuts_centroids['geometry'])
dfg = (origs.reset_index().set_index('destination_nuts').join(nuts_centroids['geometry'])
            .rename(columns={'geometry': 'dest_geom'}).dropna(how='any'))
dfg.index.name = 'destination_nuts'
dfg = dfg.reset_index().set_index(['origin_nuts', 'destination_nuts'])
df1 = dfg['orig_geom'].to_frame().reset_index()
df2 = dfg['dest_geom'].to_frame().reset_index()

df1['index'] = df1['origin_nuts'] + '-' + df1['destination_nuts'] + '-1'
df2['index'] = df1['origin_nuts'] + '-' + df1['destination_nuts'] + '-2'


df = pd.concat([df1, df2]).sort_values('index')
#df['geometry'] = df['orig_geom'].fillna(df['dest_geom'].values)
df['orig_geom'].update(df['dest_geom'])

df = df.rename(columns={'orig_geom': 'geometry'}).drop(columns=['dest_geom'])
df = gpd.GeoDataFrame(df, geometry=df.geometry).set_index('index')

df = df.set_crs(epsg=epsgs['proj']).to_crs(epsg=4326)
df['lat'] = gpd.GeoSeries(df['geometry']).y
df['lng'] = gpd.GeoSeries(df['geometry']).x


df['color'] = (df.reset_index()['index'].values)
df['color'] = df['color'].str[0:-2]

df = pd.merge(df, vols, left_on=['origin_nuts', 'destination_nuts'], right_index=True)

In [ ]:
px.line_mapbox(df, lat='lat', lon='lng', mapbox_style='carto-positron')

In [ ]:
df

In [ ]:
from shapely.geometry import LineString

vols = dff.groupby(['origin_nuts', 'destination_nuts'])['loaded_weight_kg'].sum().to_frame()

vols = (vols.join(nuts_centroids['geometry'], on=['origin_nuts']).rename(columns={'geometry': 'orig_geom'})
            .join(nuts_centroids['geometry'], on=['destination_nuts']).rename(columns={'geometry': 'dest_geom'})
            .dropna(how='any', subset=['orig_geom', 'dest_geom']))

geom = vols[['orig_geom', 'dest_geom']].apply(lambda x: LineString([x['orig_geom'], x['dest_geom']]), axis=1)
vols = gpd.GeoDataFrame(vols, geometry=geom).drop(columns=['orig_geom', 'dest_geom'])
vols = vols.set_crs(epsg=epsgs['proj']).to_crs(epsgs['world'])

df = pd.DataFrame(vols.geometry.apply(lambda x: [x.coords[0][0], x.coords[0][1], x.coords[-1][0], x.coords[-1][1]]).tolist(),
                      index=vols.index,
                      columns=['orig_lon', 'orig_lat', 'dest_lon', 'dest_lat'])
df = df.join(vols['loaded_weight_kg'])
df


GREEN_RGB = [0, 255, 0, 40]
RED_RGB = [240, 100, 0, 40]

# Specify a deck.gl ArcLayer
arc_layer = pdk.Layer(
    "ArcLayer",
    data=df,
    get_width="loaded_weight_kg / 10000",
    get_source_position=["orig_lon", "orig_lat"],
    get_target_position=["dest_lon", "dest_lat"],
    get_tilt=15,
    get_source_color=GREEN_RGB,
    get_target_color=RED_RGB,
    pickable=True,
    auto_highlight=True,
)

view_state = pdk.ViewState(latitude=38.095071, longitude=23.394901, bearing=45, pitch=50, zoom=8,)

TOOLTIP_TEXT = {"html": "loaded_weight_kg: {loaded_weight_kg}"}
r = pdk.Deck(arc_layer, initial_view_state=view_state, tooltip=TOOLTIP_TEXT, map_style='light')
r

In [ ]:
DATA_URL = "https://raw.githubusercontent.com/ajduberstein/sf_public_data/master/bay_area_commute_routes.csv"
# A bounding box for downtown San Francisco, to help filter this commuter data
DOWNTOWN_BOUNDING_BOX = [
    -122.43135291617365,
    37.766492914983864,
    -122.38706428091974,
    37.80583561830737,
]


def in_bounding_box(point):
    """Determine whether a point is in our downtown bounding box"""
    lng, lat = point
    in_lng_bounds = DOWNTOWN_BOUNDING_BOX[0] <= lng <= DOWNTOWN_BOUNDING_BOX[2]
    in_lat_bounds = DOWNTOWN_BOUNDING_BOX[1] <= lat <= DOWNTOWN_BOUNDING_BOX[3]
    return in_lng_bounds and in_lat_bounds


df = pd.read_csv(DATA_URL)
# Filter to bounding box
df = df[df[["lng_w", "lat_w"]].apply(lambda row: in_bounding_box(row), axis=1)]

GREEN_RGB = [0, 255, 0, 40]
RED_RGB = [240, 100, 0, 40]

# Specify a deck.gl ArcLayer
arc_layer = pdk.Layer(
    "ArcLayer",
    data=df,
    get_width="S000 * 2",
    get_source_position=["lng_h", "lat_h"],
    get_target_position=["lng_w", "lat_w"],
    get_tilt=15,
    get_source_color=RED_RGB,
    get_target_color=GREEN_RGB,
    pickable=True,
    auto_highlight=True,
)

view_state = pdk.ViewState(latitude=37.7576171, longitude=-122.5776844, bearing=45, pitch=50, zoom=8,)


TOOLTIP_TEXT = {"html": "{S000} jobs <br /> Home of commuter in red; work location in green"}
r = pdk.Deck(arc_layer, initial_view_state=view_state, tooltip=TOOLTIP_TEXT)
r.show()

In [ ]:
mask = (df['origin_country'] == 'EL') & (df['origin_port']== 'GRGPA')
dff = df[mask]

origs = dff[['origin_nuts', 'loaded_weight_kg']].set_index('origin_nuts').squeeze()
dests = dff[['destination_nuts', 'unloaded_weight_kg']].set_index('destination_nuts').squeeze()
dfp = nuts.copy()

dfo = dfp.join(origs).rename(columns={'loaded_weight_kg': 'value'})
dfd = dfp.join(dests).rename(columns={'unloaded_weight_kg': 'value'})
dfd['value'] *= -1

dfp = pd.concat([dfo, dfd])
bbox = dfp.total_bounds
center = {'lat': (bbox[1]+bbox[3])/2, 'lon': (bbox[0]+bbox[2])/2}
dfp = dfp.dropna(subset=['value'], how='all')

fig = px.choropleth_mapbox(dfp, 
                           geojson=dfp.geometry,
                           locations=dfp.index,
                           color='value',
                           color_continuous_scale=px.colors.diverging.RdBu_r,
                           center=center,
                           height=1000, width=1000,
                           zoom=3,
                           mapbox_style='carto-positron')
fig

In [ ]:
fig = px.scatter_mapbox(ports, lat=ports.geometry.y, lon=ports.geometry.x,
                        color='country', mapbox_style='carto-positron')
fig

In [ ]:
dfg = df.groupby(['area_code_origin', 'area_code_destination'])[['load_weight_kg', 'unload_weight_kg']].sum()

In [ ]:
col = 'load_weight_kg'

nuts_orig = 'EL302'
v = dfg.loc[nuts_orig, col]

bounds = nuts.copy()
bounds = bounds.join(v).dropna(subset=col)

bbox = bounds.total_bounds
center = {'lat': (bbox[1]+bbox[3])/2, 'lon': (bbox[0]+bbox[2])/2}

fig = px.choropleth_mapbox(bounds, 
                           geojson=bounds.geometry,
                           locations=bounds.index,
                           color=col,
                           color_continuous_scale='reds',
                           center=center,
                           zoom=3,
                           mapbox_style='carto-positron')
fig